# Spark Advanced - Bucketing

## Мотивация

Для выполнения распределенных операций JOIN необходимо запустить Shuffle операцию, которая положит на один и тот же воркер строки с одинаковым ключом. Выполнение перемешивания (shuffle) является основным препятствием на пути повышения производительности. Логически можно предположить, что необходимо заранее положить однаковые строки рядом, чтобы избежать этапа перемешивания. Второй тяжелой операцией во время соединения таблиц является сортировка. Поэтому было бы логично хранить строки в отсортированном порядке. Бакетинг призван решить эти вопросы: пользователь тратит немного больше времени на организацию хранилища, но при этом получает прирост производительности запросов на чтение.

На картинке ниже представлена схема из пяти бакетов:

![](../imgs/spark-buckets.drawio.svg)

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, sum, hash, expr
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("bucketing")
        .master("local[4]")
        .config("spark.sql.sources.bucketing.enabled", True)
        .config("spark.sql.autoBroadcastJoinThreshold", -1)
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .getOrCreate()
)
sc = spark.sparkContext

## Подготовка данных

In [ ]:
! cd /tmp && rm -rf steam && rm -rf /tmp/spark-warehouse && unzip ~/work/data/steam.zip

In [ ]:
def dump_parquet(df: DataFrame, num: int, loc: str):
    (df
        .repartition(num)
        .write
        .mode("overwrite")
        .parquet(loc)
    )

In [ ]:
sc.setJobDescription("Разбить датафреймы steam на 4 партиции")

dump_parquet(spark.read.parquet("/tmp/steam/details.parquet"), 4, "/tmp/steam_partitions/details")
dump_parquet(spark.read.parquet("/tmp/steam/tags.parquet"), 4, "/tmp/steam_partitions/tags")

В датафрейме `games` поле `app_id` имеет тип `int`, а в датафрейме `details` поле `app_id` имеет тип `long`, поэтому необходимо изменить тип колонки `app_id` в `games`:

In [ ]:
sc.setJobDescription("Поменять тип app_id с `int` на `long` в `games`")

df = (
  spark
    .read
    .parquet("/tmp/steam/games.parquet")
    .withColumn("app_id", col("app_id").cast("long"))
)
dump_parquet(df, 4, "/tmp/steam_partitions/games")

In [ ]:
sc.setJobDescription("Загрузка датафреймов steam")

games_df = spark.read.parquet("/tmp/steam_partitions/games")
details_df = spark.read.parquet("/tmp/steam_partitions/details")
tags_df = spark.read.parquet("/tmp/steam_partitions/tags")

In [ ]:
! cd /tmp && rm -rf taxi.parquet && rm -rf /tmp/spark-warehouse && unzip ~/work/data/taxi.zip

In [ ]:
sc.setJobDescription("Разбить датафрейм taxi на 4 партиции")

dump_parquet(spark.read.parquet("/tmp/taxi.parquet"), 4, "/tmp/taxi_partitions")

In [ ]:
sc.setJobDescription("Загрузка датафреймов taxi")

taxi_df = spark.read.parquet("/tmp/taxi_partitions")

## Подготовка bucketing

Только таблицы могут быть бакетированы, а информация о бакетах хранится в каталоге кластера. При работе с таблицами Catalyst может использовать информацию о бакетах, чтобы отсекать ненужные для сканирования данные.

> **Только таблицы знают о бакетах, датафреймам эта информация недоступна**

Функция `bucketing` позволит создать таблицу, записи которой будут разложены по бакетам:

In [ ]:
def bucketing(df: DataFrame, table_name: str, column: str, buckets: int = 100) -> DataFrame:
    (
    df.write
      .mode("overwrite")
      .bucketBy(buckets, column)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

**ВАЖНО:** Функция `bucketing` возвращает датафрейм, который основан на таблице. Работа с датафреймом на базе таблицы сделает информацию о бакетах нижележащей таблице доступной оптимизатору Catalyst.

На картинке ниже представлена логика работы функции `bucketing`:

- датафрейм `my_df` из двух партиций превращется в таблицу `my_table`,
- таблица `my_table` состоит из пяти бакетов,
- в одном бакете два файла: по одному на каждую партицию датафрейма `my_df`,
- в каждом файле бакета присутствует локальная сортировка.

![](../imgs/spark-dataframe-to-bucketed-table.drawio.svg)

In [ ]:
sc.setJobDescription("Создание таблиц для steam датафреймов")

games = bucketing(games_df, "games", "app_id")
details = bucketing(details_df, "details", "app_id")

In [ ]:
sc.setJobDescription("Создание таблицы для taxi")

taxi = bucketing(taxi_df, "taxi", "passenger_count")

## Базовые понятия

### Номер бакета

Проверить в каком бакете окажется строка можно при помощи функции [`pmod`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pmod.html):

In [ ]:
sc.setJobDescription("Получить распределение строк датафрейма `details_df` по бакетам")

details_buckets_pdf = (
  details_df
    .withColumn("hash", hash(col("app_id")))
    .withColumn("bucket", expr("pmod(hash, 10)"))
    .groupBy("bucket").count()
    .orderBy("bucket")
    .toPandas()
)
details_buckets_pdf.set_index("bucket", inplace=True)
_ = details_buckets_pdf.plot.pie(y="count", autopct='%1.1f%%', legend=False)

В результате видно, что строки равномерно распределны по бакетам, т.к. по `app_id` нет перекоса в `details_df`. 

Но если в датафрейме есть перекос, как например в `taxi_df` по колонке `passenger_count`, то можно заметить, что большая часть значений попадает в один бакет:

In [ ]:
sc.setJobDescription("Получить распределение по бакетам строк датафрейма `taxi_df`")

taxi_buckets_pdf = (
  taxi_df
    .withColumn("hash", hash(col("passenger_count")))
    .withColumn("bucket", expr("pmod(hash, 10)"))
    .groupBy("bucket").count()
    .orderBy("bucket")
    .toPandas()
)
taxi_buckets_pdf.set_index("bucket", inplace=True)
_ = taxi_buckets_pdf.plot.pie(y="count", autopct='%1.1f%%', legend=False)

Функция [`pmod`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pmod.html) принимает на вход два параметра: хеш и количество ожидаемых бакетов, а на выходе выдает, в каком бакете окажется запись.

Функция [`pmod`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pmod.html) не зависит от таблицы или датафрейма, а значит любые строки из разных таблиц окажутся в одном и том же бакете, если значение хеша у них совпадает.

In [ ]:
sc.setJobDescription("Демонстрация pmod")

spark.sql("select pmod(15, 5) id").show()

На картике ниже схематически изображен выбор бакета:

![](../imgs/spark-bucket-number.drawio.svg)

### Бакетирована ли таблица

Если нужно проверить бакетирована ли таблица, то можно посмотреть статистику по таблице, и в выводе будет указана информация о бакетировании:

- `Num Buckets` - сколько бакетов,
- `Bucket Columns` - по какой колонке выполнялось бакетирование.

In [ ]:
spark.sql("DESCRIBE EXTENDED details").show(200, False)

Для примера ниже создается новая таблица на базе `details_df` датафрейма, и в статистике нет информации о бакетировании, что говорит об отсутствии бакетов для таблицы:

In [ ]:
sc.setJobDescription("Создать временную таблицу `details_temp_table`")

details_df.write.mode("overwrite").saveAsTable("details_temp_table")
spark.sql("DESCRIBE EXTENDED details_temp_table").show(200, False)
_ = spark.sql("DROP TABLE details_temp_table")

#### Задание

Проверить на сколько бакетов и по какой колонке выполнялось бакетирование таблицы `taxi`.

## Использование бакетов

### Исключение партиций

Таблица `details` бакетирована по `app_id`, поэтому можно ожидать, что будет выполняться отсечение бакетов при сканировании.

Для сравнения ниже приведен план запроса для `details_df`, который ничего не знает про бакеты:

In [ ]:
sc.setJobDescription("Без бакетов: Выбрать строки по app_id = 1869690 из `details_df`")

df = details_df.where("app_id = 1869690")
df.explain(mode="formatted")

Как видно из плана запроса, применилась оптимизация `PushDownFilters`: сканирование только нужных данных с диска

In [ ]:
df.show(10, 100)

В случае работы с таблицей `details`, для которой создан отдельный датафрейм `details`, план запроса так же показывает, что отсечения по бакетам не произошло:

In [ ]:
sc.setJobDescription("С бакетами: Выбрать строки по app_id = 1869690 из details")

df = details.where(col("app_id") == F.lit(1869690).cast(LongType()))
df.explain(mode="formatted")

Это связано с правилом [`DisableUnnecessaryBucketedScan`](https://github.com/apache/spark/blob/f1bc0f938162485a96de5788f53f9fa4fb37a3b1/sql/core/src/main/scala/org/apache/spark/sql/execution/bucketing/DisableUnnecessaryBucketedScan.scala#L77C8-L77C39), Catalyst решил, что есть боле быстрый способ получения данных.

In [ ]:
df.show(10, 100)

### Снижение shuffle операций в join

В полной красе использование бакетов раскрывается при JOIN запросах.

При объединении датафрейма с таблицей, по полю, по которому таблица разбита на бакеты, shuffle будет присутствовать только на стороне датафрейма:

In [ ]:
sc.setJobDescription("Соединение бакетированной таблицы и голого датафрейма: games_df + details")

df = games_df.join(details, "app_id")
df.explain()

Такой тип объединения/join называется **one-side shuffle-free join**

In [ ]:
df.show(5)

### Join без shuffle

Но полностью получить преимущества бакетов в JOIN запросах можно, если соединение двух бакетированных таблиц выполняется по колонке бакетирования. В плане исполнения узел `Exchange` будет отсутствовать:

In [ ]:
sc.setJobDescription("Join без shuffle: details + games по app_id")

df = details.join(games, "app_id")
df.explain()

В плане запроса нет традиционного `Exchange` узла, который обычно присутствует перед операцией `Sort` при `Sort Merge Join` стратегии.

В дополнении к привычному плану можно увидеть инструкцию `SelectedBucketsCount: 100 out of 100`, которая показывает какие бакеты были просканированы. В данном случае были просканированы все 100 бакетов.

![](../imgs/spark-buckets-join.drawio.svg)

Производительность запроса можно поднять еще выше, если добавить условие по бакетированной колонке, так можно сразу исключить ненужные бакеты из анализа:

In [ ]:
sc.setJobDescription("Исключение бакетов в Join без shuffle: details + games по app_id == 1869690")

df = (
  games
    .join(details, "app_id")
    .where(col("app_id") == 1869690)
)

df.explain()

В плане запроса можно увидеть инструкцию `SelectedBucketsCount: 1 out of 100`, т.е. был просканирован только один бакет из ста. Строки с одинаковым значением ключа бакетирования попадут в один и тот же бакет. Номер бакета можно вычислить заранее, а значит нет необходимости сканировать остальные бакеты.

![](../imgs/spark-buckets-join-filter.drawio.svg)

Для сравнения можно выполнить аналогичный запрос, но уже на голых датафреймах, которые ничего не знают о бакетах:

In [ ]:
sc.setJobDescription("Join датафреймов details_df + games_df по app_id == 1869690 (только PushedFilters)")

df = (
  details_df
    .join(games_df, "app_id")
    .where(col("app_id") == 1869690)
)

df.explain()

Бакеты так же будут использоваться и при **HASH JOIN**:

In [ ]:
sc.setJobDescription("HASH JOIN: нет shuffle и сканируется только один бакет")

df = (
  details.hint("SHUFFLE_HASH")
    .join(games, "app_id")
    .where(col("app_id") == 1869690)
)
df.explain()

В плане видно:

- отсутствие узла `Exchange`,
- сканирование только одного бакета, в котором могут быть интересующие строки.

Бакеты игнорируются при использовании `BroadcastHashJoin` в силу особенностей работы алгоритма:

In [ ]:
sc.setJobDescription("BROADCAST JOIN: бакеты не используются")

df = (
  details.hint("BROADCAST")
    .join(games, "app_id")
    .where(col("app_id") == 1869690)
)
df.explain()

### Сортировка

При использовании `Sort Merge Join` по прежнему можно видеть узел `Sort` в плане запроса. От этой операции так же можно избавиться при помощи предварительной сортировки строк в бакетах. Но и тут есть свои особенности.

Функция `bucketing_single_partition` создает таблицу на базе датафрейма с одной партицией:

In [ ]:
def bucketing_single_partition(df: DataFrame, table_name: str, column: str, buckets: int = 100) -> DataFrame:
    (
    df.repartition(1)
      .write
      .mode("overwrite")
      .bucketBy(buckets, column)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

In [ ]:
sc.setJobDescription("Создание таблиц steam из датафреймов с одной партицией")

details_single_partition = bucketing_single_partition(details_df, "details_single_partition", "app_id")
games_single_partition = bucketing_single_partition(games_df, "games_single_partition", "app_id")
tags_single_partition = bucketing_single_partition(tags_df, "tags_single_partition", "tag")

In [ ]:
spark.sql("DESC EXTENDED details_single_partition").show(40, False)

In [ ]:
spark.sql("DESC EXTENDED games_single_partition").show(40, False)

In [ ]:
sc.setJobDescription("Соединение details + games: Сортировка присуствует")

details.join(games, "app_id").explain()

In [ ]:
sc.setJobDescription("Соединение details_single_partition + games_single_partition: Сортировка присуствует")

details_single_partition.join(games_single_partition, "app_id").explain()

Кажется, что независимо от того сколько партиций в датафрейме, от сортировки нет эффекта: узел `Sort` как был в плане, так и остался. Но не все так просто.

До версии 3.0 Apache Spark действительно использовал подготовленную сортировку в бакетах, но в версии 3.0 эту возможность спрятали за настройкой `spark.sql.legacy.bucketedTableScan.outputOrdering`, и теперь она отключена по умолчанию ([подробности](https://github.com/apache/spark/pull/25328)):

In [ ]:
spark.conf.get("spark.sql.legacy.bucketedTableScan.outputOrdering")

Если включить эту настройку, то из плана запроса пропдают узлы `Sort`:

In [ ]:
sc.setJobDescription("Соединение details + games: Сортировка отсуствует")

spark.conf.set("spark.sql.legacy.bucketedTableScan.outputOrdering", True)

details_single_partition.join(games_single_partition, "app_id").explain()

Но это действует только на датафреймы с одной партицией, если в датафрейме больше одной партиции, то сортировка остается:

In [ ]:
sc.setJobDescription("Соединение details + games: Сортировка присуствует")

spark.conf.set("spark.sql.legacy.bucketedTableScan.outputOrdering", True)

details.join(games, "app_id").explain()

#### Почему так?

Это связано с тем, что одна партиция будет разбита на `N` бакетов, где `N` - общее число бакетов таблицы. Если указать 100 бакетов, то для датафрейма из четырех партиций создастся 400 файлов: по четыре файла на бакет, по одному файлу на партицию.

Сортировка сохраняется только в пределах одного файла, глобальной сортировки на весь бакет нет.

![](../imgs/spark-global-sorting.drawio.svg)

С другой стороны, если один бакет состоит из одного файла, а это возможно только, если весь оригинальный датафрейм находился в одной партиции, то можно переиспользовать эту сортировку:

![](../imgs/spark-single-file-bucket-sorting.drawio.svg)

#### Зачем тогда вообще сортировка?

<details>
    <summary>А, правда, зачем?</summary>

    Сортировка позволяет исползовать бинарный поиск при получении строк из бакета
</details>

### Аггрегаты без shuffle

Преимущества отсечения данных при сканировании бакетов можно также получить при работе с агрегатами:

![](../imgs/spark-bucket-aggregation.drawio.svg)

In [ ]:
sc.setJobDescription("Агрегаты с бакетами: details.count")

details.where("app_id == 123").groupBy("app_id").count().explain()

В плане выше можно увидеть, что Catalyst использует бакеты `SelectedBucketsCount: 1 out of 100`, чего не наблюдается при использовании голых датафреймов:

In [ ]:
sc.setJobDescription("Агрегаты без бакетов: details_df.count")

details_df.where("app_id == 123").groupBy("app_id").count().explain()

### Оконные функции без shuffle

Оконные функции также получают преимущества от бакетов:

In [ ]:
sc.setJobDescription("Оконные функции: details.count")

df = (
  details
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('app_id'))
    )
    .where(col("app_id") == 123)
  )
df.explain()

In [ ]:
sc.setJobDescription("Оконные функции: details_single_partition.count")

df = (
  details_single_partition
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('app_id'))
    )
    .where(col("app_id") == 123)
  )
df.explain()

In [ ]:
sc.setJobDescription("Оконные функции: details_df.count")

df = (
  details_df
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('app_id'))
    )
    .where(col("app_id") == 123)
  )
df.explain()

## Особенности

Бакеты дают огромный прирост производительности, но иногда бакеты могут не активироваться, поэтому важно понимать особенности их работы.

### Неравное количество бакетов

Не всегда есть возможность разбить таблицы на одинаковое число бакетов. При неравном числе бакетов таблиц, участвующих в соединении, Apache Spark может все еще воспользоваться бакетами, но есть ограничения.

Apache Spark при помощи алгоритма `coalesce` может соединить несколько бакетов в один, но выполнять он это будет попарно. Следовательно, если таблицы разбиты на неравное число бакетов, то меньшее число бакетов должно быть делителем большего числа бакетов. В этом случае Apache Spark может снизить бОльшее число бакетов до меньшего числа бакетов, а дальше воспользоваться обычным алгоритмом работы с бакетами.

![](../imgs/spark-coalesce-buckets.drawio.svg)

Таблица `details` будет иметь 25 бакетов:

In [ ]:
sc.setJobDescription("Число бакетов details == 25")

details = bucketing(details_df, "details", "app_id", 25)

In [ ]:
spark.sql("DESC EXTENDED details").where("col_name == 'Num Buckets'").show()

Таблица `games` состоит из 100 бакетов:

In [ ]:
spark.sql("DESC EXTENDED games").where("col_name == 'Num Buckets'").show()

25 является делителем 100, поэтому Apache Spark должен свободно воспользоваться бакетами:

In [ ]:
sc.setJobDescription("Слияние бакетов недоступно: games + details")

(
games
  .join(details, "app_id")
  .explain()
)

Но на практике этого не выходит почему-то...

Также не получается воспользоваться бакетами, если явно поменять число партиций на 25:

In [ ]:
sc.setJobDescription("Слияние бакетов недоступно: изменение числа партиций games")

(
games.repartition(25, "app_id")
  .join(details, "app_id")
  .explain()
)

Бакеты и партиции разные вещи, поэтому явное изменение числа партиций просто создало новый датафрейм, у которого 25 партиций, он уже никакого отношения не имеет к таблице, которая разбита на 100 бакетов.

Механизм автоматического слияния бакетов для достижения однакого числа бакетов в соединяемых таблицах контролируется настройкой `spark.sql.bucketing.coalesceBucketsInJoin.enabled`, которая выключена по умолчанию:

In [ ]:
spark.conf.get("spark.sql.bucketing.coalesceBucketsInJoin.enabled")

После ее включения Spark должен начать склеивать партиции:

In [ ]:
sc.setJobDescription("Слияние бакетов недоступно: games + details (bug)")

spark.conf.set("spark.sql.bucketing.coalesceBucketsInJoin.enabled", True)

(
games
  .join(details, "app_id")
  .explain()
)

Но этого не происходит из-за [ошибки в логике](https://issues.apache.org/jira/browse/SPARK-43021) оптимизатора: склеивание бакетов не происходит при включенном AQE:

In [ ]:
sc.setJobDescription("Слияние бакетов работает: games + details (без AQE)")

# автоматическое склеивание партиций не работает при включенном AQE. Обновления в https://github.com/apache/spark/pull/40688
spark.conf.set("spark.sql.adaptive.enabled", False)
(
games
  .join(details, "app_id")
  .explain()
)

В плане выше можно заметить:

- операции `Exchange` отсутствуют,
- бакеты таблицы `games` были склеены по два так, что получилось в итоге 25 бакетов `SelectedBucketsCount: 100 out of 100 (Coalesced to 25)`.

Полное отключение AQE - плохая идея, поэтому после запроса необхдимо включить его назад:

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", True)

Не любое число бакетов может быть склеено. Склеивание будет выполняться только, если отншение бОльшего числа бакетов к меньшему не превышает порогового значения указанного в `spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio`:

In [ ]:
spark.conf.get("spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio")

По умолчанию используется значение 4. Сейчас отношение числа бакетов `games` к числу бакетов `details` 2:1, поэтому склеивание произошло успешно:

![](../imgs/spark-many-coalesce-buckets.drawio.svg)

Если указать 2 в качества порогового значения, то склеивание бакетов выполняться не будет, $log_2(2) = 1$, т.е. можно склеить бакеты один раз, а нужно два раза: $100 \rightarrow 50 \rightarrow 25$:

In [ ]:
sc.setJobDescription("Слияние бакетов недоступно: maxBucketRatio превышен")

spark.conf.set("spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio", 2)

# автоматическое склеивание партиций не работает при включенном AQE. Обновления в https://github.com/apache/spark/pull/40688
spark.conf.set("spark.sql.adaptive.enabled", False)
(
games
  .join(details, "app_id")
  .explain()
)

spark.conf.set("spark.sql.adaptive.enabled", True)
spark.conf.set("spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio", 4)

В плане снова можно увидеть узел `Exchange`.

Бакеты можно также использовать и в Spark SQL запросах:

In [ ]:
sc.setJobDescription("Слияние бакетов работает: Spark SQL")

# автоматическое склеивание партиций не работает при включенном AQE. Обновления в https://github.com/apache/spark/pull/40688
spark.conf.set("spark.sql.adaptive.enabled", False)

spark.sql("select /*+ SHUFFLE_HASH(u) */ * from games join details using (app_id)").explain()
spark.conf.set("spark.sql.adaptive.enabled", True)

### Загрузка данных

Spark хранит метаинформацию о том, что конкретная таблица имеет конкретное количество бакетов, поэтому необходимо создавать датафрейм из таблицы:

In [ ]:
sc.setJobDescription("Загрузка данных: загрузка таблицы")

df = (
  spark
    .table("details")
    .groupBy("app_id")
    .count()
)
df.explain()

При этом если загрузить данные просто как parquet, то бакетинг использоваться не будет:

In [ ]:
sc.setJobDescription("Загрузка данных: чтение parquet")

df = (
  spark
    .read.parquet('/tmp/spark-warehouse/details')
    .groupby('app_id')
    .count()
)
df.explain()

### Разные типы данных ключа

Одной из самых неприятных проблем, с которой можно столкнуться - это разница типов ключей бакетирования в соединяемых таблицах: Apache Spark не будет автоматически приводить `int` к `long` или `double`, а просто откажется использовать бакеты.

В качестве примера ниже создается датафрейм, в котором `app_id` имеет тип `int`:

In [ ]:
sc.setJobDescription("Поменять тип `app_id`: long -> int")

games_df_int_key = (
  games_df
    .withColumn(
      "app_id",
      col("app_id").cast("int")
    )
)
games_df_int_key.printSchema()

На базе датафрейма `games_df_int_key` создается новая бакетированная таблица:

In [ ]:
sc.setJobDescription("Создание таблицы `games_int_key`, бакетированной по `app_id`")

games_int_key = bucketing(games_df_int_key, "games_int_key", "app_id")

Запрос ниже показывает, что бакеты для новой таблицы активировались:

In [ ]:
sc.setJobDescription("Проверка бакетов таблицы `games_int_key`")

games_int_key.groupBy("app_id").count().explain()

Но если соединить таблицу `games_int_key` c таблицей `details`, то в результате можно увидеть узлы `Exchange`, что говорит о том, что Apache Spark не использует бакеты полноценно:

In [ ]:
sc.setJobDescription("Соединение games_int_key + details: бакеты не используются")

games_int_key.join(details, "app_id").explain()

Решением будет изменение типа колонки `app_id` в `games_int_key` на `long` или типа колонки `app_id` в `details` на `int`

### Пользовательские функции

In [ ]:
sc.setJobDescription("Вернуть схему бакетирования для `details`")

details = bucketing(details_df, "details", "app_id")

Пользовательские функцию могут препятствовать использованию бакетов.

Например, ниже определена булевая фукнция, которая определяет много или мало отзывов у игры:

In [ ]:
many_reviews = lambda reviews: reviews > 5
many_reviews_udf = F.udf(many_reviews)

Если применить эту функцию к таблице перед соединением, то информация о бакетах не используется, и оптимизатор добавляет узел `Exchange` в план запроса:

In [ ]:
sc.setJobDescription("UDF мешает бакетам")

(
  games.withColumn(
      "many_reviews",
      many_reviews_udf(col("user_reviews"))
    )
    .join(details, "app_id")
    .explain()
)

Решением будет применение функции после соединения таблиц:

In [ ]:
sc.setJobDescription("Бакеты работают с UDF")

(
  games
    .join(details, "app_id")
    .withColumn(
      "many_reviews",
      many_reviews_udf(col("user_reviews"))
    )
    .explain()
)

### Динамическое партиционирование

Бакеты используются с высококардинальными признаками/колонками. Если колонка может содержать небольшое число различных значений, то лучше подойдет динамическое партционирование. Партиционирование можно комбинировать с бакетами:

![](../imgs/spark-partitioning.drawio.svg)

Для демонстрации динамического партиционирования создается новый датафрейм `games_part_by_year_df` на базе датафрейма `games_df`, который будет партиционирован по году релиза игры:

In [ ]:
sc.setJobDescription("Динамическое партиционирование: создать `games_part_by_year_df` с колонкой `year`")

games_part_by_year_df = games_df \
  .withColumn(
    "year",
    F.date_format(col("date_release"), "y").cast("int")
  )

Функция `bucketing_partition` разбивает датафрейм на партиции, а каждая партиция еще дополнительно разбивается на бакеты:

In [ ]:
def bucketing_partition(df: DataFrame, table_name: str, column: str, partition: str, buckets: int = 100) -> DataFrame:
    (
    df.write
      .mode("overwrite")
      .partitionBy(partition)
      .bucketBy(buckets, column)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

In [ ]:
sc.setJobDescription("Создание таблицы `games_part_by_year`")

games_part_by_year = bucketing_partition(
    games_part_by_year_df,
    "games_part_by_year",
    "app_id",
    "year"
)

При запросе данных из партиционированной таблицы по ключу партиционирования, оптимизатор применит правило `PartitionFilters` при сканировании таблицы:

In [ ]:
sc.setJobDescription("Динамическое партиционирование: PartitionFilers по колонке `year`")

df = games_part_by_year.where('year == 2012')
df.explain(mode="formatted")

### Обновление данных в таблице

В таблицу `games_part_by_year` будут добавлены новые записи за следующий год:

In [ ]:
sc.setJobDescription("Получить максимальный год в `games_part_by_year`")

[[year]] = games_part_by_year.select(F.max("year")).collect()

In [ ]:
year += 1

In [ ]:
sc.setJobDescription("Подготовить новые данные для `games_part_by_year`")

new_data = (
  games_part_by_year
    .where(col('year') == year - 10)
    .withColumn('date_release', F.add_months(col('date_release'), 10 * 12))
    .withColumn('year', F.lit(year))
    .limit(5)
  )

new_data.explain()

Вставить подготовленные данные в `games_part_by_year`

In [ ]:
sc.setJobDescription("Вставить подготовленные данные в `games_part_by_year`")

new_data.write.insertInto("games_part_by_year")

In [ ]:
sc.setJobDescription("Получить вставленные данные")

df = games_part_by_year.where(col("year") == year)
df.explain(mode="formatted")

In [ ]:
df.count()

### Немного математики

В итоге в каждой партиции количество файлов определяется формулой:
$$
Количество\spaceфайлов\spaceв\spaceпартиции = Количество\spaceбакетов \times Количество\spaceпартиций\spaceоригинального\spaceдатафрейма
$$

**Дано:**

- количество партиций в датафрейме `games_part_by_year_df`: 4;
- количество бакетов: 100.

Тогда число файлов в партиции будет равно 400

In [ ]:
!find /tmp/spark-warehouse/games_part_by_year/*2015* -type f -name "*.parquet" | wc -l

А всего файлов для итоговой таблицы будет в $|X|$ раз больше, где $|X|$ мощность множества колонки партиционирования (количество уникальных значений):

In [ ]:
!find /tmp/spark-warehouse/games_part_by_year/ -type f -name "*.parquet" | wc -l

Эту особенность нужно учитывать, т.к. сканирование большого числа небольших файлов может сильно снизить производительность запроса.

### Конфигурация параллельного сканирования файлов

Сканирование файлов с диска будет выполняться параллельно, если число файлов превышает пороговое значение (по умолчанию 32). Пороговое значение можно настраивать:

- [`spark.sql.sources.parallelPartitionDiscovery.threshold`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L1621) - сколько файлов в директории активирует параллельный режим обхода директории и запускает задачу обхода директории на кластере,
- [`spark.sql.sources.parallelPartitionDiscovery.parallelism`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L1633) - максимальныое число заданий (task) при чтении данных. Желательно настраивать, чтобы заданий (tasks) не было слишком много.

## Конфигурация

| Настройка | Описание | Значение по умолчанию |
| --------- | -------- | --------------------- |
| [`spark.sql.sources.bucketing.enabled`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L1483) | Включить поддержку бакетов | `True` |
| [`spark.sql.sources.bucketing.maxBuckets`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L1545) | Максимальное количество бакетов на таблицу | `100 000` (100 тысяч) |
| [`spark.sql.sources.bucketing.autoBucketedScan.enabled`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L1553) | Исключить использование бакетов из плана запроса, если без них эффективнее | `True` |
| [`spark.sql.bucketing.coalesceBucketsInJoin.enabled`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L4120) | Если две таблицы при join имеют разное количество бакетов, слить (coalesce) несколько бакетов большей таблицы в один, чтобы уровнять число бакетов. Сработает только, если меньшее число бакетов является делителем большего (100 и 50, например) | `False` |
| [`spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L4133) | Во сколько раз сколько количество бакетов может отличаться, чтобы выполнить слияние согласно настройке `spark.sql.bucketing.coalesceBucketsInJoin.enabled` (см. выше). Число шагов слияния будет не больше $log_2(spark.sql.bucketing.coalesceBucketsInJoin.enabled)$ | `4` (не больше чем в 4 раза) |
| [`spark.sql.legacy.bucketedTableScan.outputOrdering`](https://github.com/apache/spark/blob/834d71b990468006ae4b1df17fae31f639d0d7ff/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L3623) | Исключить сортировку из плана запроса, если бакет состоит из одного файла, и все данные в нем отсортированы | `False` |

## Вывод

При классической OLAP нагрузке, где данные записываются один раз, а считываются много раз, использование бакетов может значительно увеличить производительность запросов. Но, как и многие техники оптимизации, бакетирование не является универсальным способом, подходящим для любых нагрузок. К основным недостаткам можно отнести:

- появление большого числа файлов,
- ограниченность типа запросов.

Много файлов само по себе может и не быть проблемой, но если файлы имеют небольшой размер, то для HDFS это может стать большой проблемой. NameNode может хранить ограниченное число файлов, хотя и очень большое, что диктует максимальный объем хранилища.

При создании бакетированной таблицы, очень важно выполнять запросы по ключу бакетирования, чтобы отсекать ненужные файлы заранее. В противном случае, для запросов без ключа бакетирования Apache Spark будет вынужен запускать сканирование всех файлов таблицы. Одно из решений - создание отдельной таблицы со своим ключом бакетирования для каждого типа запроса. Но нужно быть внимательным, т.к. в таком случае данные будут дублироваться в нескольких таблицах. Синхронизация данных относится техникам организации озера данных (Data Lake), которое будет разбираться позднее.

<details>
    <summary><b>Spoiler Alert</b></summary>

    Для синхронизации данных в таблицах можно использовать Structured Streaming, и это единственное приемлемое применение Structured Streaming.
</details>

## Задания

1. На базе датафрейма `tags_df` создать таблицу `tags` с ключом бакетирования `tag`;
1. На базе датафрейма `details_df` создать таблицу `details_by_tag` с ключом бакетирования `tag`;
1. Выполнить join между `details_by_tag` и `tags` по полю `tag`, убедиться что бакеты используются;
1. Посчитать количество строк по каждому тегу в таблице с деталями, проанализировать план;
1. Составить наиболее оптимальный join между таблицами с тегами, деталями и играми. Подсказка: возможно понадобяться дополнительные колонки.